What datasets are supported?

IXI and CASILab

How hard would it be to include a new dataset?

~2 days if like IXI or CASILab

What files are excluded from each dataset?



What preprocessing is ran?

How are the labels received?

What do I need to do to make it so that dementia score can be used instead? -> Nothing

In [ ]:
#@title Installations, downloads, imports, settings
!pip install --upgrade --quiet ipdb
!pip install --upgrade --quiet nibabel #==3.1.0
!pip install --upgrade --quiet nilearn #==0.5.2
!pip install --upgrade –-quiet SimpleITK
!pip install --upgrade --quiet substring
!pip install --upgrade --quiet deprecated
!pip install --upgrade --quiet torchio
!curl -s -o colormap.txt https://raw.githubusercontent.com/thenineteen/Semiology-Visualisation-Tool/master/slicer/Resources/Color/BrainAnatomyLabelsV3_0.txt
!pip install --quiet cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl
# %xmode Verbose
import abc
import ipdb
import os
import sys
import numpy as np
import pandas as pd
import nibabel as nib
# from nibabel.affines import rescale_affine
# if not os.path.exists('/usr/local/lib/python3.6/dist-packages/nibabel_320'):
#   !pip install 'nibabel>=3.2.0'
# import nibabel_320 as nib
import nilearn
from nilearn import plotting
try:
  import niwidgets
except:
  print("Not importing niwidgets since not installed")
import SimpleITK as sitk
import tempfile
import unittest
import time
import datetime
import substring
import shlex
import pathlib
from pathlib import Path
import enum
from deprecated import deprecated
from google.colab import auth, drive
from tqdm.notebook import tqdm
import multiprocessing
import torch
from torch import nn
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.distributed.parallel_loader as pl
import torch_xla.debug.metrics as met
import torchio as tio
from torchio import DATA
from torchio.transforms import (
  RandomFlip,
  RandomAffine,
  RandomElasticDeformation,
  RandomNoise,
  RandomMotion,
  RandomBiasField,
  RescaleIntensity,
  Resample,
  ToCanonical,
  ZNormalization,
  CropOrPad,
  HistogramStandardization,
  OneOf,
  Compose,
)
import tensorflow as tf
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
print(tf.__version__)
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Input, Conv3D, Dropout, Activation, MaxPooling3D, Flatten, Dense
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
%load_ext tensorboard
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import warnings
sns.set_style("whitegrid", {'axes.grid' : False})
%config InlineBackend.figure_format = 'retina'
np.random.seed(0)

     |████████████████████████████████| 788kB 7.4MB/s 
     |████████████████████████████████| 368kB 28.4MB/s 
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.18 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.23.1 which is incompatible.
     |████████████████████████████████| 3.3MB 6.8MB/s 
     |████████████████████████████████| 3.1MB 6.8MB/s 
ERROR: Invalid requirement: '–-quiet'
     |████████████████████████████████| 153kB 6.5MB/s 
     |████████████████████████████████| 42.5MB 2.5MB/s 
ERROR: torch_xla-1.7-cp36-cp36m-linux_x86_64.whl is not a supported wheel on this platform.


/usr/local/lib/python3.7/dist-packages/nilearn/datasets/__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


Not importing niwidgets since not installed


ModuleNotFoundError: ignored

Where to retrieve data

What datasets

Which files/Which model

What *hyperparameters

In [ ]:
datasets_to_run_with = {'IXI', 'CASILab'}

In [ ]:
move_tfrecords_to_cloud = False #@param {type: "boolean"}
use_cloud_tfrecords = False #@param {type: "boolean"}
# TODO: Have to use gcsfuse?
use_gcsfuse_tfrecords = False #@param {type: "boolean"}
use_google_drive_stored_tfrecords = not use_cloud_tfrecords and not use_gcsfuse_tfrecords
run_model = True #@param {type: "boolean"}
make_estimator = True # Delete? Must be true for now # is this true???
use_crescenta_tpu = True #Del;ete/?

In [ ]:
#@title Filenames, Paths, user input

class Filenames:
  TFRECORD_EXTENSION = '.tfrecord'
  NIFTI_IMAGE_EXTENSION = '.nii'
  METAIMAGE_EXTENSION = '.mha'

  data = 'Data'
  archives = 'Archives'
  pytorch = 'PyTorch'
  tensorflow = 'TensorFlow'
  logs = 'logs2'
  model = 'model'
  casilab = 'CASILab'
  casilab_labels = 'NormalDatabase2.xlsx'
  ixi = 'IXI'
  ixi_labels = 'Demographinc info_IXI.xls'
  mra = 'MRA'
  raw = 'Raw'
  nearest_resampling_to_128x128x50 = 'Nearest Resampling to 128x128x50'
  augmented1 = 'Augmented1'
  metaimages = 'MetaImages'
  niftis = 'Niftis'
  tfrecords = 'TFRecords'

  def has_tfrecord_extension(filename):
    return len(filename) >= len(Filenames.TFRECORD_EXTENSION) and filename[-len(Filenames.TFRECORD_EXTENSION):] == Filenames.TFRECORD_EXTENSION

  def add_potential_numeric_suffix(filepath):
    if not os.path.exists(filepath):
      return filepath
    number_to_suffix_with = 1
    while os.path.exists(filepath + "-" + str(number_to_suffix_with)):
      number_to_suffix_with += 1
    return filepath + "-" + str(number_to_suffix_with)

  def convert_to_new_extension(filename, new_extension):
    try:
      return substring.substringByChar(filename, None, ".")[:-1] + new_extension
    except:
      return filename + new_extension


class FilenamesTestCase(unittest.TestCase):
  root_dir = 'testing/'

  def setUp(self):
    os.mkdir(FilenamesTestCase.root_dir)

  def tearDown(self):
    os.rmdir('testing/')

  def test_add_potential_numeric_with_no_existing_file(self):
    original_name = FilenamesTestCase.root_dir + 'test_file.extension'
    self.assertEqual(Filenames.add_potential_numeric_suffix(original_name),
                     original_name)

class Paths:
  mri = Path('/content/drive/My Drive/AI/MRI')

  archives = mri/Filenames.archives
  archives_pytorch = archives/Filenames.pytorch
  archives_tensorflow = archives/Filenames.tensorflow
  logs = archives/Filenames.logs
  model = archives/Filenames.model

  data = mri/Filenames.data
  casilab = data/Filenames.casilab
  casilab_labels = casilab/Filenames.casilab_labels
  casilab_mra = casilab/Filenames.mra
  casilab_mra_raw = casilab_mra/Filenames.raw
  casilab_mra_raw_metaimages = casilab_mra_raw/Filenames.metaimages
  casilab_mra_raw_niftis = casilab_mra_raw/Filenames.niftis
  casilab_mra_raw_tfrecords = casilab_mra_raw/Filenames.tfrecords
  casilab_mra_nearest_resampling_to_128x128x50_niftis = casilab_mra/Filenames.nearest_resampling_to_128x128x50/Filenames.niftis
  casilab_mra_nearest_resampling_to_128x128x50_tfrecords = casilab_mra/Filenames.nearest_resampling_to_128x128x50/Filenames.tfrecords
  ixi = data/Filenames.ixi
  ixi_mra = ixi/Filenames.mra
  ixi_labels = ixi/Filenames.ixi_labels
  ixi_mra_raw = ixi_mra/Filenames.raw
  ixi_mra_raw_niftis = ixi_mra_raw/Filenames.niftis
  ixi_mra_raw_tfrecords = ixi_mra_raw/Filenames.tfrecords
  ixi_mra_nearest_resampling_to_128x128x50_niftis = ixi_mra/Filenames.nearest_resampling_to_128x128x50/Filenames.niftis
  ixi_mra_nearest_resampling_to_128x128x50_tfrecords = ixi_mra/Filenames.nearest_resampling_to_128x128x50/Filenames.tfrecords

  def get_all_paths_in_directory(directory_path):
    for filename in Path.iterdir(directory_path):
      yield os.path.join(directory_path, filename)

  def get_equivalent_path_with_part_replaced(path, original_part, new_part):
    path = Path(path)
    path_parts = list(path.parts)
    for index, part in enumerate(path_parts):
      if part == original_part:
        path_parts[index] = new_part
    return Path(*path_parts)

  def run_check_on_path(path, ask=False):
    path = Path(path)
    dir = path if path.is_dir() else path.parent
    if not dir.exists():
      if not ask or UserInput.yes_or_no("Do you want to create the directories needed to make " + str(dir)):
        try:
          dir.mkdir(parents=True)
        except FileExistsError:
          print(f'Directory {dir} was made already in the mean time')
    elif ask and len(list(dir.iterdir())) > 0:
      if UserInput.yes_or_no('There are files in ' + str(dir) + '. Do you want the program to exit?'):
        sys.exit()

class UserInput:
  def yes_or_no(question):
    reply = str(input(question+' (y/n): ')).lower().strip()
    if reply[0] == 'y':
        return True
    if reply[0] == 'n':
        return False
    else:
        return yes_or_no("Uhhhh... please enter ")

NameError: ignored

##Storage

In [ ]:
class Storage(enum.Enum):
  GOOGLE_DRIVE = enum.auto()
  GOOGLE_CLOUD = enum.auto()
  USING_GCSFUSE = enum.auto()

###Google Drive (DELETE?)

In [ ]:
class GoogleDrive:
  drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title If using Google Cloud
if use_cloud_tfrecords:
  class GoogleCloud:

In [ ]:
#@title Dataset super class
class Dataset(abc.ABC):
  @abc.abstractmethod
  def get_dataframe(self):
    ...

  @abc.abstractmethod
  def get_path_to_image_data(self):
    ...

  def get_list_of_file_age_pairs(self):
    list_of_file_age_pairs = []
    for row_index, series in self.get_dataframe().iterrows():
      list_of_file_age_pairs.append((series["FILE PATH"], series["AGE"]))
    return list_of_file_age_pairs

  def get_new_tfrecord_file_path(self, filename):
    new_tfrecord_file_path = self.get_tfrecords_path() / Path(substring.substringByChar(filename, None, ".")[:-1]).with_suffix(Filenames.TFRECORD_EXTENSION)
    return new_tfrecord_file_path

    #Add more to this

In [ ]:
#@title CASILab
class CASILabDataset(Dataset):
  LEN_OF_ID = 3
  string_before_id_in_example_name = "NormalA"
  start_of_id_in_example_name = len(string_before_id_in_example_name)
  end_of_id_in_example_name = start_of_id_in_example_name + LEN_OF_ID - 1

  string_before_id_in_filename = "Normal"
  string_after_id_in_filename = "-MRA"
  start_of_id_in_filename = len(string_before_id_in_filename)
  end_of_id_in_filename = start_of_id_in_filename + LEN_OF_ID - 1

  example_to_not_use = 48
  image_shape_for_examples_1_through_95 = (448, 448, 128)
  image_shape_for_examples_96_through_99 = (352, 448, 176)

  def __init__(self):
    self.examples_to_skip = [CASILabDataset.example_to_not_use]
    self.dataframe = self._get_and_initialize_dataframe()
    self.number_of_examples = self.dataframe.shape[0]

  def _is_an_example_name(string):
    try:
      return substring.substringByInd(string, 0, CASILabDataset.start_of_id_in_example_name - 1) == CASILabDataset.string_before_id_in_example_name
    except:
      return False

  def _get_casilab_id_from_example_name(example_name):
    return substring.substringByInd(example_name, CASILabDataset.start_of_id_in_example_name, CASILabDataset.end_of_id_in_example_name)
    
  def _is_an_example_to_not_use(self, string):
    return int(CASILabDataset._get_casilab_id_from_example_name(string)) in self.examples_to_skip
    
  def _is_a_row_to_drop(self, row):
    example_name = row['EXAMPLE NAME']
    age = row['AGE']
    return type(example_name) != str or \
        not CASILabDataset._is_an_example_name(example_name) or \
        self._is_an_example_to_not_use(example_name) or pd.isna(age)

  def _get_indices_to_drop(self, dataframe):
    indexes_to_drop = []
    for i in range(dataframe.shape[0]):
      row = dataframe.iloc[i]
      if self._is_a_row_to_drop(row):
        indexes_to_drop.append(i)
    return indexes_to_drop
  
  def _get_casilab_id_from_file_path(file_path):
    return substring.substringByInd(file_path.name, CASILabDataset.start_of_id_in_filename, CASILabDataset.end_of_id_in_filename)
  
  def _get_casilab_id_as_int_from_file_path(file_path):
    return int(_get_casilab_id_from_file_path(file_path))

  def _get_and_initialize_dataframe(self):
    dataframe = pd.read_excel(Paths.casilab_labels, header=None)
    dataframe = dataframe.rename(mapper={0: "EXAMPLE NAME", 6: "AGE"}, axis=1)
    dataframe = dataframe.drop(self._get_indices_to_drop(dataframe)).reset_index(drop=True)
    file_paths_for_dataframe = [None] * dataframe.shape[0]

    for index, example_name in dataframe["EXAMPLE NAME"].iteritems():
      for file_path in self.get_path_to_image_data().iterdir():
        if CASILabDataset._get_casilab_id_from_example_name(example_name) == CASILabDataset._get_casilab_id_from_file_path(file_path):
          file_paths_for_dataframe[index] = file_path
    dataframe.insert(loc=1, column="FILE PATH", value=file_paths_for_dataframe)

    return dataframe

  def get_dataframe(self):
    return self.dataframe
    
  def get_number_of_examples(self):
    return self.dataframe.shape[0]

  def get_file_path_by_index(self, example_index):
    return self.dataframe["FILE PATH"][example_index]

  def get_casilab_id_as_string(casilab_id):
    return str(casilab_id).zfill(3)

  def get_nifti_file_path(casilab_id):
    return CASILabDataset.string_before_id_in_filename + CASILabDataset.get_casilab_id_as_string(casilab_id) + CASILabDataset.string_after_id_in_filename + Filenames.NIFTI_IMAGE_EXTENSION

  def get_age(self, example_index):
    return self.dataframe["AGE"][example_index]

  def get_path_to_image_data(self):
    return Paths.casilab_mra_raw_niftis

  def get_path_to_niftis_nearest_resampled_to_128x128x50():
    return Paths.casilab_mra_nearest_resampling_to_128x128x50_niftis

  def get_nifti_file_path(self, casilab_id):
    return os.path.join(CASILabDataset.get_path_to_(), CASILabDataset.get_nifti_filename(casilab_id))

  def get_tfrecords_path(self):
    return Paths.casilab_mra_raw_tfrecords

In [ ]:
#@title IXI
class IXIDataset(Dataset): 
  LEN_OF_ID = 3
  example_to_not_use = 371
  image_shape_from_guys_and_hammersmith_hospital = (512, 512, 100)
  image_shape_from_institute_of_psychiatry = (1024, 1024, 92)

  def __init__(self):
    self.examples_to_skip = [IXIDataset.example_to_not_use]
    self.id_start_index=3
    self.dataframe = self._get_and_initialize_dataframe()
    self.number_of_examples = self.dataframe.shape[0]

  def _get_indices_to_drop(self, dataframe):
    return dataframe.index[np.logical_or(np.logical_or(
                             dataframe['IXI_ID'] == IXIDataset.example_to_not_use,
                             pd.isna(dataframe['AGE'])), 
                             pd.isna(dataframe['FILE PATH']))].tolist()

  
  def _get_IXI_id(self, file_path):
    ixi_id_string = file_path.name[self.id_start_index:self.id_start_index + IXIDataset.LEN_OF_ID]
    ixi_id = int(ixi_id_string)
    return ixi_id

  def _get_and_initialize_dataframe(self):
    dataframe = pd.read_excel(Paths.ixi_labels)
    number_of_ixi_ids_in_paper = dataframe['IXI_ID'].size
    file_paths_for_dataframe = [None] * number_of_ixi_ids_in_paper
    for index, element in dataframe['IXI_ID'].iteritems():
      for file_path in self.get_path_to_image_data().iterdir():
        if element == self._get_IXI_id(file_path):
          file_paths_for_dataframe[index] = file_path
    dataframe.insert(loc=1, column='FILE PATH', value=file_paths_for_dataframe)
    dataframe = dataframe.drop(self._get_indices_to_drop(dataframe)).reset_index(drop=True)

    return dataframe

  def get_dataframe(self):
    return self.dataframe

  def get_file_path_by_index(self, example_index):
    return self.dataframe['FILE PATH'][example_index]

  def get_age(self, example_index):
    return self.dataframe['AGE'][example_index]

  def get_path_to_image_data(self):
    return Paths.ixi_mra_raw_niftis

  def get_path_to_niftis_nearest_resampled_to_128x128x50():
    return Paths.ixi_mra_nearest_resampling_to_128x128x50_niftis

  def get_tfrecords_path(self):
    return Paths.ixi_mra_raw_tfrecords

In [ ]:
#@title Serialization
class Serialization:
  def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
      value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

  def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

  def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

  def serialize_example(image_array, age, affine_array=None):
    image_string = Serialization.serialize_numpy_array(image_array)
   
    feature = {
        'image': Serialization._bytes_feature(image_string),
        'age': Serialization._float_feature(age),
    }

    if affine_array is None:
      affine_string = Serialization.serialize_numpy_array(affine_array)
      feature['affine'] = Serialization._bytes_feature(affine_string)

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

  def serialize_numpy_array(array):
    tensor = tf.convert_to_tensor(array, dtype=tf.float32)
    return tf.io.serialize_tensor(tensor)

  def serialize_nifti_image_array(image):
    array = np.asarray(image)
    return Serialization.serialize_numpy_array(array)

In [ ]:
#@title Affines
class Affines:
  shape = (4, 4)

##Data

In [ ]:
class Data:
  def write_tfrecord(tfrecord_file_path, serialization):
    writer = tf.io.TFRecordWriter(str(tfrecord_file_path))
    writer.write(serialization)
    writer.close()

  def convert_nifti_to_tfrecord(nifti_file_path, tfrecord_file_path):
    nifti_image = nib.load(nifti_file_path)
    nifti_image = Data.reshape_nifti_image_for_model(nifti_image)
    image_array = nifti_image.get_fdata()
    serialized_example = Serialization.serialize_example(image_array, age, affine_array=nifti_image.affine)
    Data.write_tfrecord(tfrecord_filepath, serialized_example)

  def convert_niftis_to_tfrecords(niftis_path, tfrecords_path=None):
    for nifti_filename in os.listdir(niftis_path):
      niftis_filepath = os.path.join(niftis_path, nifti_filename)
      parent_filepath, _ = os.path.split(niftis_path)
      tfrecord_file_path = os.path.join(parent_filepath, Filenames.convert_to_new_extension(nifti_filename, Filenames.TFRECORD_EXTENSION))
      Data.convert_nifti_to_tfrecord(niftis_filepath, tfrecord_file_path)
  
  def create_tfrecords_dataset_from_nifti_dataset(dataset):
    Paths.run_check_on_path(dataset.get_tfrecords_path())
    for example_index in range(dataset.number_of_examples):
      filename = dataset.get_filename_by_index(example_index)
      age = dataset.get_age(example_index)
      if filename != None and pd.notna(age):
        tfrecord_filepath = dataset.get_new_tfrecord_file_path(filename)
        if not os.path.exists(tfrecord_filepath):
          nifti_file_path = dataset.get_path_to_image_data()/filename
          nifti_image = nib.load(nifti_file_path)
          image_array = nifti_image.get_fdata()
          serialized_example = Serialization.serialize_example(image_array, age, affine_array=nifti_image.affine)
          Data.write_tfrecord(tfrecord_filepath, serialized_example)

  def create_tfrecords_datasets_from_nifti_datasets(datasets=[CASILabDataset(), IXIDataset()]):
    for dataset in datasets:
      Data.create_tfrecords_dataset_from_nifti_dataset(dataset)

  def write_subjects_dataset_to_nifti_files(subjects_dataset):
    for subject in subjects_dataset:
        subject_image = subject.get_first_image()
        subject_image_array = np.squeeze(subject_image.numpy())
        nifti_image = nib.Nifti1Image(subject_image_array, subject_image.affine)
        nib.save(nifti_image, subject.output_path)

  def get_subjects_dataset(dataset, transform=None):
    subjects = []
    source_data = type(dataset).get_path_to_niftis_nearest_resampled_to_128x128x50()
    new_niftis_path = Paths.get_equivalent_path_with_part_replaced(source_data, Filenames.nearest_resampling_to_128x128x50, Filenames.augmented1)
    Paths.run_check_on_path(new_niftis_path)
    for image_path in source_data.iterdir():
      new_nifti_file_path = Paths.get_equivalent_path_with_part_replaced(image_path, Filenames.nearest_resampling_to_128x128x50, Filenames.augmented1)
      if not new_nifti_file_path.exists():
        subject = tio.Subject(mri=tio.ScalarImage(image_path), output_path=new_nifti_file_path)
        subjects.append(subject)
    subjects_dataset = tio.SubjectsDataset(subjects, transform=transform)
    return subjects_dataset

  def create_augmented_datasets(training_dataset=IXIDataset(), test_dataset=CASILabDataset()):
    training_transform = Compose([
      # ToCanonical(),
      # Resample(4),
      # CropOrPad((48, 60, 48), padding_mode='reflect'),
      # RandomMotion(),
      # HistogramStandardization({'mri': landmarks}),
      RandomBiasField(),
      # ZNormalization(masking_method=ZNormalization.mean),
      # RandomNoise(),
      # RandomFlip(axes=(0,)),
      # OneOf({
      #     RandomAffine(): 0.8,
      #     RandomElasticDeformation(): 0.2,
      # }),
    ])

    # validation_transform = Compose([
    #   # ToCanonical(),
    #   Resample(4),
    #   CropOrPad((48, 60, 48), padding_mode='reflect'),
    #   HistogramStandardization({'mri': landmarks}),
    #   ZNormalization(masking_method=ZNormalization.mean),
    # ])

    training_subjects_dataset = Data.get_subjects_dataset(training_dataset, transform=training_transform)
    # test_subjects = Data.get_subjects_dataset(test_dataset)
    Data.write_subjects_dataset_to_nifti_files(training_subjects_dataset)

  def get_target_affine(nifti_image, target_shape):
    shape = nifti_image.get_fdata().shape
    zooms = np.diag(nifti_image.affine)[:3] * np.divide(shape, target_shape)
    return rescale_affine(nifti_image.affine, shape, zooms, new_shape=target_shape)
  
  def reshape_nifti_image_for_model(nifti_image):
    target_affine = Data.get_target_affine(nifti_image, AgePredictor.image_shape)
    return nilearn.image.resample_img(nifti_image, target_affine=target_affine, target_shape=AgePredictor.image_shape, interpolation='nearest')

  def create_reshaped_nifti_from_nifti(nifti_file_path, reshaped_nifti_file_path):
    nifti_image = nib.load(nifti_file_path)
    reshaped_nifti_image = Data.reshape_nifti_image_for_model(nifti_image)
    Paths.run_check_on_path(nifti_file_path)
    nib.save(nifti_image, reshaped_nifti_file_path)

  def create_reshaped_niftis_from_niftis(niftis_path):
    Paths.run_check_on_path(niftis_path) 
    for nifti_file_path in Paths.get_all_paths_in_directory(niftis_path):
      Data.create_reshaped_nifti_from_nifti(nifti_file_path, Paths.get_equivalent_path_with_part_replaced(nifti_file_path, Filenames.raw, Filenames.nearest_resampling_to_128x128x50))

  def create_reshaped_niftis_from_niftis_for_datasets(datasets=[IXIDataset()]):
    for dataset in datasets:
      Data.create_reshaped_niftis_from_niftis(dataset.get_path_to_image_data())

  def convert_metaimage_to_nifti1image(metaimage_file_path, nifti1image_file_path):
    image = sitk.ReadImage(metaimage_file_path)
    sitk.WriteImage(image, nifti1image_file_path)

  def convert_metaimages_to_nifti1images(metaimage_path, nifti1image_path):
    if metaimage_path[-1] != '/':
      metaimage_path += '/'
    if nifti1image_path[-1] != '/':
      nifti1image_path += '/'
    for filename in os.listdir(metaimage_path):
      filename_without_extension = substring.substringByChar(filename, None, '.')[:-1]
      metaimage_file_path = metaimage_path + filename
      nifti1image_file_path = nifti1image_path + filename_without_extension + Filenames.NIFTI_IMAGE_EXTENSION
      if not os.path.exists(nifti1image_file_path):
        Data.convert_metaimage_to_nifti1image(metaimage_file_path, nifti1image_file_path)

  def move_directory_from_google_drive_to_google_cloud(directory_path):
    directory_shell_path = shlex.quote(directory_path)
    !gsutil cp -m file://{directory_shell_path}* {GoogleCloud.bucket_contents_uri}

  def get_dataset_size(dataset):
    return len(os.listdir(dataset.get_tfrecords_path()))

In [ ]:
#@title Plotting
class Plotting:
  casilab_dataset = CASILabDataset()
  def plot_nifti_image_using_niwidgets(nifti_image):
    image = niwidgets.NiftiWidget(nifti_image)
    image.nifti_plotter()

  def plot_nifti_image_using_nilearn(nifti_image):
    plotting.plot_img(nifti_image)
    # nilearn.plotting.plot_anat(nifti_image, cut_coords=None, display_mode='ortho')

  def plot_nifti_image(nifti_image):
    Plotting.plot_nifti_image_using_nilearn(nifti_image)

  def plot_nifti_file(nifti_file_path):
    nifti_image = nib.load(nifti_file_path)
    Plotting.plot_nifti_image(nifti_image)

  def plot_casilab_nifti_file(casilab_id):
    nifti_file_path = casilab_dataset.get_nifti_file_path(casilab_id)
    Plotting.plot_nifti_file(nifti_file_path)

  def plot_nifti_image_reshaped(nifti_image):
    nifti_image_with_target_shape = Data.reshape_nifti_image_for_model(nifti_image)
    Plotting.plot_nifti_image(nifti_image_with_target_shape)

  def plot_nifti_file_reshaped(nifti_file_path):
    nifti_image = nib.load(nifti_file_path)
    Plotting.plot_nifti_image_reshaped(nifti_image)

  def plot_casilab_nifti_file_reshaped(casilab_id):
    nifti_file_path = casilab_dataset.get_nifti_file_path(casilab_id)
    Plotting.plot_nifti_file_reshaped(nifti_file_path)

  def plot_original_and_reshaped_version_of_casilab_nifti_file(casilab_id):
    print("Original version of first casilab nifti file")
    Plotting.plot_casilab_nifti_file(casilab_id)
    print("Reshaped version of first casilab nifti file")
    Plotting.plot_casilab_nifti_file_reshaped(casilab_id)

  def plot_mri_scan(image_array, affine):
    try:
      nifti_image = nib.Nifti1Image(image_array, affine)
    except:
      nifti_image = nib.Nifti2Image(image_array, affine)
    plot_nifti_image(nifti_image)

  def tensorboard():
    logdir = shlex.quote(str(Paths.logs))
    %tensorboard --logdir {logdir}

NameError: ignored

In [ ]:
#@title Visualization functions from torchIO_tutorial.ipynb
# https://colab.research.google.com/github/fepegar/torchio-notebooks/blob/main/notebooks/TorchIO_tutorial.ipynb#scrollTo=QS_PG3sMsH4w
def show_nifti(image_path_or_image, colormap='gray'):
    try:
        from niwidgets import NiftiWidget
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=FutureWarning)
            widget = NiftiWidget(image_path_or_image)
            widget.nifti_plotter(colormap=colormap)
    except Exception:
        if isinstance(image_path_or_image, nib.AnalyzeImage):
            nii = image_path_or_image
        else:
            image_path = image_path_or_image
            nii = nib.load(str(image_path))
        k = int(nii.shape[-1] / 2)
        plt.imshow(nii.dataobj[..., k], cmap=colormap)

def show_subject(subject, image_name, label_name=None):
    if label_name is not None:
        subject = copy.deepcopy(subject)
        affine = subject[label_name].affine
        label_image = subject[label_name].as_sitk()
        label_image = sitk.Cast(label_image, sitk.sitkUInt8)
        border = sitk.BinaryContour(label_image)
        border_array, _ = tio.utils.sitk_to_nib(border)
        border_tensor = torch.from_numpy(border_array)[0]
        image_tensor = subject[image_name].data[0]
        image_tensor[border_tensor > 0.5] = image_tensor.max()
    with tempfile.NamedTemporaryFile(suffix='.nii') as f:
        subject[image_name].save(f.name)
        show_nifti(f.name)

def plot_histogram(axis, tensor, num_positions=100, label=None, alpha=0.05, color=None):
    values = tensor.numpy().ravel()
    kernel = stats.gaussian_kde(values)
    positions = np.linspace(values.min(), values.max(), num=num_positions)
    histogram = kernel(positions)
    kwargs = dict(linewidth=1, color='black' if color is None else color, alpha=alpha)
    if label is not None:
        kwargs['label'] = label
    axis.plot(positions, histogram, **kwargs)

# Visualization functions from https://colab.research.google.com/github/fepegar/torchio-notebooks/blob/main/notebooks/Data_preprocessing_and_augmentation_using_TorchIO_a_tutorial.ipynb#scrollTo=IA2LRIRc62Pr&uniqifier=2
def get_bounds(self):
    """Get image bounds in mm.

    Returns:
        np.ndarray: [description]
    """
    first_index = 3 * (-0.5,)
    last_index = np.array(self.spatial_shape) - 0.5
    first_point = nib.affines.apply_affine(self.affine, first_index)
    last_point = nib.affines.apply_affine(self.affine, last_index)
    array = np.array((first_point, last_point))
    bounds_x, bounds_y, bounds_z = array.T.tolist()
    return bounds_x, bounds_y, bounds_z

def to_pil(image):
    from PIL import Image
    from IPython.display import display
    data = image.numpy().squeeze().T
    data = data.astype(np.uint8)
    image = Image.fromarray(data)
    w, h = image.size
    display(image)
    print()  # in case multiple images are being displayed

def stretch(img):
    p1, p99 = np.percentile(img, (1, 99))
    from skimage import exposure
    img_rescale = exposure.rescale_intensity(img, in_range=(p1, p99))
    return img_rescale

def show_fpg(
        subject,
        to_ras=False,
        stretch_slices=True,
        indices=None,
        intensity_name='t1',
        parcellation=True,
        ):
    subject = tio.ToCanonical()(subject) if to_ras else subject
    def flip(x):
        return np.rot90(x)
    fig, axes = plt.subplots(2, 3, figsize=(12, 8))
    if indices is None:
        half_shape = torch.Tensor(subject.spatial_shape) // 2
        i, j, k = half_shape.long()
        i -= 5  # use a better slice
    else:
        i, j, k = indices
    bounds_x, bounds_y, bounds_z = get_bounds(subject.t1)  ###

    orientation = ''.join(subject.t1.orientation)
    if orientation != 'RAS':
        import warnings
        warnings.warn(f'Image orientation should be RAS+, not {orientation}+')
    
    kwargs = dict(cmap='gray', interpolation='none')
    data = subject[intensity_name].data
    slices = data[0, i], data[0, :, j], data[0, ..., k]
    if stretch_slices:
        slices = [stretch(s.numpy()) for s in slices]
    sag, cor, axi = slices
    
    axes[0, 0].imshow(flip(sag), extent=bounds_y + bounds_z, **kwargs)
    axes[0, 1].imshow(flip(cor), extent=bounds_x + bounds_z, **kwargs)
    axes[0, 2].imshow(flip(axi), extent=bounds_x + bounds_y, **kwargs)

    kwargs = dict(interpolation='none')
    data = subject.seg.data
    slices = data[0, i], data[0, :, j], data[0, ..., k]
    if parcellation:
        sag, cor, axi = [color_table.colorize(s.long()) if s.max() > 1 else s for s in slices]
    else:
        sag, cor, axi = slices
    axes[1, 0].imshow(flip(sag), extent=bounds_y + bounds_z, **kwargs)
    axes[1, 1].imshow(flip(cor), extent=bounds_x + bounds_z, **kwargs)
    axes[1, 2].imshow(flip(axi), extent=bounds_x + bounds_y, **kwargs)
    
    plt.tight_layout()


class ColorTable:
    def __init__(self, colors_path):
        self.df = self.read_color_table(colors_path)

    @staticmethod
    def read_color_table(colors_path):
        df = pd.read_csv(
            colors_path,
            sep=' ',
            header=None,
            names=['Label', 'Name', 'R', 'G', 'B', 'A'],
            index_col='Label',
        )
        return df

    def get_color(self, label: int):
        """
        There must be nicer ways of doing this
        """
        try:
            rgb = (
                self.df.loc[label].R,
                self.df.loc[label].G,
                self.df.loc[label].B,
            )
        except KeyError:
            rgb = 0, 0, 0
        return rgb

    def colorize(self, label_map: np.ndarray) -> np.ndarray:
        rgb = np.stack(3 * [label_map], axis=-1)
        for label in np.unique(label_map):
            mask = label_map == label
            color = self.get_color(label)
            rgb[mask] = color
        return rgb

color_table = ColorTable('colormap.txt')

##Hardware

In [ ]:
class Hardware:
  def get_strategy():
    strategy = tf.distribute.get_strategy()
    try:
      if use_crescenta_tpu:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver
      else:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
      tf.config.experimental_connect_to_cluster(resolver)
      # This is the TPU initialization code that has to be at the beginning.
      tf.tpu.experimental.initialize_tpu_system(resolver)
      strategy = tf.distribute.TPUStrategy(resolver)
      print("All devices: ", tf.config.list_logical_devices('TPU'))
    except:
      pass
    return strategy

##Dataset Info Callback

In [ ]:
class DatasetInfoCallback(keras.callbacks.Callback):
  i = 0
  def __init__(self, training_set_size, validation_set_size, test_set_size):
    self.training_set_size = training_set_size
    self.validation_set_size = validation_set_size
    self.test_set_size = test_set_size
    
  def on_train_begin(self, logs=None):
    tf.summary.text('Training set size', str(self.training_set_size), step=0)
    tf.summary.text('Validation set size', str(self.validation_set_size), step=0)
    tf.summary.text('Test set size', str(self.test_set_size), step=0)

##Action

In [ ]:
class Action(enum.Enum):
    TRAIN = 'Training'
    VALIDATE = 'Validation'
    TEST = 'Test'

##Alarm

In [ ]:
class Alarm:
  def bell():
    sys.stdout.write('\r\a')
    sys.stdout.flush()

##Age Predictor

In [ ]:
class AgePredictor(abc.ABC):
  batch_size = 16
  num_epochs = 64
  image_shape = (128, 128, 50)

  def __init__(self):
    self.start_datetime = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    self.logdir = self.get_logdir()/self.start_datetime

  @abc.abstractmethod
  def get_logdir(self):
    ...

  preprocessing_transform = Compose([
    ToCanonical(),
    Resample(1, image_interpolation='bspline'),
    CropOrPad((128, 128, 50), padding_mode='reflect'),
    #   HistogramStandardization({'mri': landmarks}),
    #   ZNormalization(masking_method=ZNormalization.mean),
  ])

  augmentation_transform = Compose([
    # RandomMotion(),
    RandomBiasField(),
    RandomNoise(),
    RandomFlip(axes=(0,)),
    # OneOf({
    #     RandomAffine(): 0.8,
    #     RandomElasticDeformation(): 0.2,
    # }),
  ])

###Age Predictor Using Torch

In [ ]:
class AgePredictorUsingTorch(AgePredictor):
  def get_logdir(self):
    return Paths.archives_pytorch

  def get_model_and_optimizer_and_loss(self):
    model = nn.Sequential(
        nn.Conv3d(1, 8, 3),
        nn.Dropout(0.2),
        nn.ReLU(),
        nn.MaxPool3d(2),

        nn.Conv3d(8, 8, 3),
        nn.Dropout(0.2),
        nn.ReLU(),
        nn.MaxPool3d(2),

        nn.Conv3d(8, 16, 3),
        nn.Dropout(0.2),
        nn.ReLU(),
        nn.MaxPool3d(2),

        nn.Flatten(),

        nn.Linear(12544, 128),
        nn.Dropout(0.3),
        nn.ReLU(),

        nn.Linear(128, 1),
        nn.Dropout(0.3),
        nn.ReLU(),
    ).to(self.device)
    optimizer = torch.optim.Adam(model.parameters(), lr=10e-5)
    loss = nn.L1Loss()
    return model, optimizer, loss
  
  def get_data_loader(self, dataset, is_distributed=True, shuffle=False, sampler=None):
    if is_distributed:
      batch_size=self.batch_size
      num_workers=self.num_workers
      drop_last=True
    else:
      batch_size=AgePredictor.batch_size
      num_workers=multiprocessing.cpu_count()
      drop_last=False

    return torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        sampler=sampler,
        num_workers=num_workers,
        drop_last=drop_last
    )

  def get_distributed_sampler(self, dataset, shuffle=False):
    return torch.utils.data.distributed.DistributedSampler(
             dataset,
             num_replicas=xm.xrt_world_size(),
             rank=xm.get_ordinal(),
             shuffle=True)

  def get_subjects_dataset(self, file_age_pairs, augment=False):
    subjects = []
    for image_path, age in file_age_pairs:
      subject = tio.Subject(mri=tio.ScalarImage(path=image_path), age=age)
      subjects.append(subject)
    transform = AgePredictor.preprocessing_transform
    if augment:
      transform = Compose([transform, AgePredictor.augmentation_transform])
    subjects_dataset = tio.SubjectsDataset(subjects, transform=transform)
    return subjects_dataset
      
  def __init__(self, device, flags=None, is_distributed=True, process_index=1):
    super().__init__()
    self.device = device
    self.process_index = process_index
    if flags != None:
      try:
        self.batch_size = flags['batch_size']
      except:
        pass
      try:
        self.num_epochs = flags['num_epochs']
      except:
        pass
      try:
        self.num_workers = flags['num_workers']
      except:
        if is_distributed:
          raise Exception("Num workers must be provided to run distributedly")

    fitting_file_age_pairs = IXIDataset().get_list_of_file_age_pairs()
    training_split_ratio = 0.9
    num_fitting_subjects = len(fitting_file_age_pairs)
    num_training_subjects = int(training_split_ratio * num_fitting_subjects)

    training_file_age_pairs = fitting_file_age_pairs[:num_training_subjects]
    validation_file_age_pairs = fitting_file_age_pairs[num_training_subjects:]
    test_file_age_pairs = CASILabDataset().get_list_of_file_age_pairs()

    # if not xm.is_master_ordinal():
    #   xm.rendezvous('all_workers_received_data')

    training_set_subjects_dataset = self.get_subjects_dataset(training_file_age_pairs, augment=True)
    validation_set_subjects_dataset = self.get_subjects_dataset(validation_file_age_pairs)
    test_set_subjects_dataset = self.get_subjects_dataset(test_file_age_pairs)

    # if xm.is_master_ordinal():
    #   xm.rendezvous('all_workers_received_data')
    # if self.process_index == 0:
    #   time.sleep(2)

    training_sampler, validation_sampler, test_sampler = None, None, None
    if is_distributed:
      training_sampler = self.get_distributed_sampler(training_set_subjects_dataset, shuffle=True)
      validation_sampler = self.get_distributed_sampler(validation_set_subjects_dataset)
      test_sampler = self.get_distributed_sampler(test_set_subjects_dataset)

    self.training_loader = self.get_data_loader(training_set_subjects_dataset, is_distributed, shuffle=not is_distributed, sampler=training_sampler)
    self.validation_loader = self.get_data_loader(validation_set_subjects_dataset, is_distributed, sampler=validation_sampler)
    self.test_loader = self.get_data_loader(test_set_subjects_dataset, is_distributed, sampler=test_sampler)

    self.model, self.optimizer, self.loss = self.get_model_and_optimizer_and_loss()

  def forward(self, inputs):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)
        prediction = self.model(inputs)
    return prediction

  def prepare_batch(self, batch):
    inputs = batch['mri'][DATA].to(self.device)
    targets = batch['age'].to(self.device)
    return inputs, targets
  
  def run_epoch(self, action, loader, is_distributed=True, epoch_idx=None):
    epoch_start = time.time()
    is_training = action == Action.TRAIN
    self.model.train(is_training)
    if is_distributed:
      loader = pl.ParallelLoader(self.training_loader, [self.device]).per_device_loader(self.device)
    epoch_losses = torch.zeros(len(loader))
    for batch_idx, batch in enumerate(loader):
        inputs, targets = self.prepare_batch(batch)
        self.optimizer.zero_grad()
        with torch.set_grad_enabled(is_training):
            predicted_age = self.forward(inputs)
            targets = targets.view_as(predicted_age)
            batch_loss = self.loss(predicted_age, targets)
            epoch_losses[batch_idx] = batch_loss
            if is_training:
                batch_loss.backward()
                if self.device.type != 'cuda' and self.device.type != 'cpu':
                  xm.optimizer_step(self.optimizer, barrier=not is_distributed)
                else:
                  self.optimizer.step()
    elapsed_epoch_time = time.time() - epoch_start
    print(f'Epoch {epoch_idx}: {action.value} mean absolute age difference: {torch.mean(epoch_losses).item():0.3f} in {elapsed_epoch_time} seconds')

  def train(self, is_distributed=True, weights_stem='whole images'):
    train_start = time.time()
    self.run_epoch(Action.VALIDATE, self.validation_loader, epoch_idx=0)
    for epoch_idx in range(1, AgePredictor.num_epochs + 1):
        print('Starting epoch', epoch_idx)
        self.run_epoch(Action.TRAIN, self.training_loader, epoch_idx=epoch_idx)
        self.run_epoch(Action.VALIDATE, self.validation_loader, epoch_idx=epoch_idx)
        file_path = f'{self.logdir/weights_stem}_epoch_{epoch_idx}.pth'
        file_path = Filenames.add_potential_numeric_suffix(file_path)
        Paths.run_check_on_path(file_path)
        save_function = xm.save if is_distributed else torch.save
        save_function(self.model.state_dict(), file_path)
    elapsed_train_time = time.time() - train_start
    print("Epoch Process", self.process_index, "finished training. Train time was:", elapsed_train_time, " seconds") 

  def test(self):
    # GEt modle from weight directory
    eval_start = time.time()
    self.run_epoch(Action.TEST, self.test_loader)
    elapsed_eval_time = time.time() - eval_start
    print("Process", self.process_index, "finished evaluation. Evaluation time was:", elapsed_eval_time, " seconds")

In [ ]:
# At global scope.
SERIAL_EXEC = xmp.MpSerialExecutor()

# import torch_xla.debug.metrics as met
def map_fn(index, flags):
  torch.manual_seed(flags['seed'])
  device = xm.xla_device()
  # if not xm.is_master_ordinal():
  #   xm.rendezvous('download_only_once')

  age_predictor_using_torch = AgePredictorUsingTorch(device, flags, process_index=index) #is this in the right place?

  # if xm.is_master_ordinal():
  #   xm.rendezvous('download_only_once')

  age_predictor_using_torch.train()
  age_predictor_using_torch.test()

def run(distributed=True):
  flags = {}
  flags['batch_size'] = AgePredictor.batch_size
  flags['num_epochs'] = AgePredictor.num_epochs
  flags['num_workers'] = 8 if os.environ.get('TPU_NAME', None) else 1
  flags['seed'] = 0
  if distributed:
    xmp.spawn(map_fn, args=(flags,), nprocs=8, start_method='fork')
  else:
    try:
      device = xm.xla_device()
    except:
      device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
    print("Not using multiprocessing, using device: ", str(device))

In [ ]:
batch_loss = torch.tensor(3)
batch_loss2 = torch.tensor(4)
epoch_losses = []
epoch_losses.append(batch_loss)
epoch_losses.append(batch_loss2)
epoch_losses = np.array(epoch_losses)

print(epoch_losses.mean())

3.5


###Age Predictor Using Tensorflow

In [ ]:
class AgePredictorUsingTensorflow(AgePredictor):
  # batch_size = 16 * tpu_strategy.num_replicas_in_sync()

  def get_logdir(self):
    return Paths.archives_tensorflow

  def _get_tfrecord_filepaths(retrieve_from, dataset):
    if retrieve_from in {Storage.GOOGLE_CLOUD, Storage.USING_GCSFUSE}:
      tfrecord_filepaths = !gsutil ls {bucket_contents_uri}
    if retrieve_from == Storage.USING_GCSFUSE:
      !echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
      !curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
      !apt -qq update
      !apt -qq install gcsfuse
      tfrecords_path = '/content/crescenta_bucket/'
      !gcsfuse {bucket_contents_uri} {tfrecords_path}
      tfrecord_filenames = [get_filename_from_path(tfrecord_filename) for tfrecord_filename in tfrecord_filepaths]
    if retrieve_from == Storage.GOOGLE_DRIVE:
      tfrecords_path = dataset.get_tfrecords_path()
    if retrieve_from in {Storage.GOOGLE_DRIVE, Storage.USING_GCSFUSE}:
      tfrecord_filepaths = tfrecords_path.iterdir()
      tfrecord_filepaths = [str(tfrecord_filepath) for tfrecord_filepath in tfrecord_filepaths]
    return tfrecord_filepaths

  def __init__(self, training_dataset=IXIDataset(), test_dataset=CASILabDataset(), retrieve_from=Storage.GOOGLE_DRIVE):
    self.file_writer = tf.summary.create_file_writer(self.logdir + "/metrics")
    self.file_writer.set_as_default()
    self.tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=self.logdir, histogram_freq=1)
    self.modeldir = os.path.join(Paths.model, start_datetime)

    self.input_shape = AgePredictor.image_shape + (1,)

    tfrecord_filepaths_for_fitting = AgePredictorUsingTensorflow._get_tfrecord_filepaths(retrieve_from, training_dataset)
    number_of_training_examples = int(len(tfrecord_filepaths_for_fitting) * .9)
    self.tfrecord_filepaths_for_training = tfrecord_filepaths_for_fitting[:number_of_training_examples]
    self.tfrecord_filepaths_for_validation = tfrecord_filepaths_for_fitting[number_of_training_examples:]
    self.tfrecord_filepaths_for_testing = AgePredictorUsingTensorflow._get_tfrecord_filepaths(retrieve_from, test_dataset)
    self.dataset_info_callback = DatasetInfoCallback(len(self.tfrecord_filepaths_for_training), len(self.tfrecord_filepaths_for_validation), len(self.tfrecord_filepaths_for_testing))
    self.input_fn_callable = lambda: self.input_fn(tfrecord_filepaths=self.tfrecord_filepaths_for_training)
    self._initialize_and_compile_keras_model()

  # def get_normalization_layer(dataset):
  #   # Create a Normalization layer for our feature.
  #   normalizer = Normalization()

  #   # Prepare a Dataset that only yields our feature.
  #   feature_ds = dataset.map(lambda x, y: x)

  #   # Learn the statistics of the data.
  #   normalizer.adapt(feature_ds)

  #   return normalizer

  def _initialize_and_compile_keras_model(self):
    # normalization_layer = AgePredictorUsingTensorflow.get_normalization_layer(self.input_fn(self.tfrecord_filepaths_for_training))

    with Hardware.get_strategy().scope():
      # inputs = Input(shape=AgePredictorUsingTensorflow.input_shape)
      # normalized_inputs = normalization_layer(inputs)
      
      sequential_part = Sequential([
        Conv3D(8, 3, input_shape=self.input_shape),
        Dropout(0.2),
        Activation('relu'),
        MaxPooling3D(),

        Conv3D(8, 3),
        Dropout(0.2),
        Activation('relu'),
        MaxPooling3D(),

        Conv3D(16, 3),
        Dropout(0.2),
        Activation('relu'),
        MaxPooling3D(),

        Flatten(),

        Dense(128),
        Dropout(0.3),
        Activation('relu'),

        Dense(1),
        Dropout(0.3),
        Activation('relu'),
      ])

      # outputs = sequential_part(normalized_inputs)

      self._keras_model = sequential_part # Model(inputs, outputs)
      self._keras_model.summary()
      self._keras_model.compile(loss='mean_absolute_error', optimizer=Adam(lr=0.0001))


  def _tranform_example(self, dictionary, augment=False):
    image_array = tf.make_ndarray(dictionary['image'])
    image_array = np.moveaxis(image_array, image_array.ndim - 1, 0) 
    image = tio.ScalarImage(tensor=image_array, affine=dictionary['affine'].numpy())
    preprocessing_transform = Compose([
      ToCanonical(),
      Resample(1),
      CropOrPad((128, 128, 50), padding_mode='reflect'),
    #   HistogramStandardization({'mri': landmarks}),
      ZNormalization(masking_method=ZNormalization.mean),
    ])

    transformed_image = preprocessing_transform(image)
    if augment:
      augmentation_transform = Compose([
        # RandomMotion(),
        RandomBiasField(),
        RandomNoise(),
        RandomFlip(axes=(0,)),
        # How about random flipping in other directions?
        # OneOf({
        #     RandomAffine(): 0.8,
        #     RandomElasticDeformation(): 0.2,
        # }),
      ])
      transformed_image = augmentation_transform(transformed_image)
    image_array = np.moveaxis(transformed_image.numpy(), 0, image_array.ndim - 1)
    image_tensor = tf.convert_to_tensor(image_array)
    return (image_tensor, dictionary['age'])

  def _make_example(self, dictionary):
    return (dictionary['image'], dictionary['age'])

  def _make_dictionary(self, features_dict):
    example_dictionary = {}
    example_dictionary['image'] = features_dict['image']    
    example_dictionary['image'] = tf.io.parse_tensor(example_dictionary['image'], out_type=tf.float32)
    example_dictionary['image'] = tf.reshape(example_dictionary['image'], list(AgePredictorUsingTensorflow.input_shape))

    try:
      example_dictionary['affine'] = features_dict['affine']
      example_dictionary['affine'] = tf.io.parse_tensor(example_dictionary['affine'], out_type=tf.float32)
      example_dictionary['affine'] = tf.reshape(example_dictionary['affine'], list(Affines.shape))
    except:
      pass

    example_dictionary['age'] = features_dict['age']

    return example_dictionary

  def _parse(features_proto, get_affine=True): 
    feature_description = {
      'image': tf.io.FixedLenFeature([], tf.string),
      'age': tf.io.FixedLenFeature([], tf.float32),
    }
    if get_affine:
      feature_description['affine'] = tf.io.FixedLenFeature([], tf.string)
    return tf.io.parse_single_example(features_proto, feature_description)

  def input_fn(self, tfrecord_filepaths, shuffle=False, augment=False):
    ipdb.set_trace()
    AUTO = tf.data.experimental.AUTOTUNE
    serialized_dataset = tf.data.TFRecordDataset(tfrecord_filepaths, num_parallel_reads=AUTO)    
    parsed_dataset = serialized_dataset.map(AgePredictorUsingTensorflow._parse, num_parallel_calls=AUTO)
    dictionaries_dataset = parsed_dataset.map(self._make_dictionary, num_parallel_calls=AUTO)
    examples_dataset = dictionaries_dataset.map(self._make_example, num_parallel_calls=AUTO)
    # numpy_dataset = tfds.examples_dataset
    # transform_example_callable = lambda dictionary: self._tranform_example(dictionary, augment=augment)
    # examples_dataset = examples_dataset.map(transform_example_callable, num_parallel_calls=AUTO)
    if shuffle:
      examples_dataset = examples_dataset.shuffle(len(examples_dataset))
    batched_dataset = examples_dataset.batch(AgePredictorUsingTensorflow.batch_size)
    batched_dataset = batched_dataset.prefetch(buffer_size=AUTO)
    return batched_dataset

  def train_keras_model(self):
    self._keras_model.fit(x=self.input_fn(self.tfrecord_filepaths_for_training, shuffle=True, augment=True), epochs=self.num_epochs, callbacks=[self.tensorboard_callback, self.dataset_info_callback], validation_data=self.input_fn(self.tfrecord_filepaths_for_validation))
    self._keras_model.save(Filenames.add_potential_numeric_suffix(self.modeldir))

  def get_latest_model_dir():
    return os.path.join(Paths.model, os.listdir(Paths.model)[-1])

  def load_most_recent_runs_weights_into_keras_model(self):
    latest_model_dir = AgePredictorUsingTensorflow.get_latest_model_dir()
    self._keras_model = load_model(latest_model_dir)

  def evaluate_keras_model(self):
    self.load_most_recent_runs_weights_into_keras_model()
    self._keras_model.evaluate(x=self.input_fn(self.tfrecord_filepaths_for_testing), batch_size=AgePredictorUsingTensorflow.batch_size, callbacks=[self.tensorboard_callback])

  def predict_with_keras_model(self, tfrecord_filepaths):
    self._keras_model.predict(x=self.input_fn(tfrecord_filepaths), batch_size=1, callbacks=[self.tensorboard_callbacks])
    
  def _initialize_estimator(self):
    model_dir = tempfile.mkdtemp()
    self.estimator = tf.keras.estimator.model_to_estimator(keras_model=self._keras_model, model_dir=model_dir)

  ''' For estimators, train, evaluate, and predict functions, probably need to break up the dataset per each'''
  def train_estimator(self):
    self.estimator.train(input_fn=self.input_fn_callable, steps=AgePredictorUsingTensorflow.num_epochs)
  
  def evaluate_estimator(self):
    eval_result = self.estimator.evaluate(input_fn=self.input_fn_callable)
    print('Eval result: {}'.format(eval_result))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Main

In [ ]:
if __name__ == '__main__':
  try:
    use_torch = True
    if use_torch:
      # age_predictor_using_torch = AgePredictorUsingTorch()
      # age_predictor_using_torch.train()
      # age_predictor_using_torch.test()
      run()
    else:
      unittest.main(argv=['first-arg-is-ignored'], exit=False)
      Plotting.tensorboard()
      # age_predictor = AgePredictor()
      # age_predictor.train_keras_model()
      # age_predictor.evaluate_keras_model()
  except Exception as e:
    Alarm.bell()
    raise e
  

Epoch 0: Validation mean absolute age difference: 47.841 in 286.5001928806305 seconds
Epoch 0: Validation mean absolute age difference: 48.614 in 280.83963537216187 seconds
Epoch 0: Validation mean absolute age difference: 49.505 in 280.5566363334656 seconds
Epoch 0: Validation mean absolute age difference: 52.061 in 286.3459038734436 seconds
Epoch 0: Validation mean absolute age difference: 47.414 in 280.8190610408783 seconds
Starting epoch 1
Starting epoch 1
Starting epoch 1
Starting epoch 1
Starting epoch 1
Epoch 0: Validation mean absolute age difference: 49.452 in 300.39104747772217 seconds
Starting epoch 1
Epoch 0: Validation mean absolute age difference: 46.985 in 303.5924117565155 seconds
Starting epoch 1
Epoch 0: Validation mean absolute age difference: 46.248 in 317.868816614151 seconds
Starting epoch 1
Epoch 1: Training mean absolute age difference: 50.752 in 351.05918049812317 seconds
Epoch 1: Training mean absolute age difference: 43.259 in 358.91727232933044 seconds
Epoch

#Test Ground

# References

Code references:
[TPUs in Colab](https://colab.research.google.com/notebooks/tpu.ipynb#scrollTo=LtAVr-4CP1rp)

Data references:
http://brain-development.org/ixi-dataset/,
http://insight-journal.org/midas/community/view/21
